# Setup

In [1]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.abspath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [3]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [4]:
import pprint

pprint.pprint(
    [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
)

[{'_datacenter': 'dc1',
  '_rack': 'rack1',
  'broadcast_address': '10.15.20.2',
  'broadcast_port': 7001,
  'broadcast_rpc_address': '10.15.20.2',
  'broadcast_rpc_port': 9041,
  'conviction_policy': <cassandra.policies.SimpleConvictionPolicy object at 0x000001A43D056BA0>,
  'dse_version': None,
  'dse_workload': None,
  'dse_workloads': None,
  'endpoint': <DefaultEndPoint: cassandra-node-1.mavasbel.vpn.itam.mx:9041>,
  'host_id': UUID('3feb94c8-8f0a-4b13-b78e-76dcae54f085'),
  'is_up': True,
  'listen_address': '172.21.0.2',
  'listen_port': 7001,
  'lock': <unlocked _thread.RLock object owner=0 count=0 at 0x000001A43AE84D40>,
  'release_version': '5.0.6'},
 {'_datacenter': 'dc1',
  '_rack': 'rack1',
  'broadcast_address': '10.15.20.2',
  'broadcast_port': 7002,
  'broadcast_rpc_address': '10.15.20.2',
  'broadcast_rpc_port': 9042,
  'conviction_policy': <cassandra.policies.SimpleConvictionPolicy object at 0x000001A43D045810>,
  'dse_version': None,
  'dse_workload': None,
  'dse_wo

In [5]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [6]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [7]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [8]:
from faker import Faker

fake = Faker()

In [9]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [10]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [11]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'Lake Jacobtown', 'user_id': UUID('92886505-24aa-438b-8cae-770be6e38ee6'), 'last_access': datetime.datetime(2026, 1, 10, 2, 4, 24, 679000), 'session_duration': 2310, 'username': 'sandovalandrea'}
{'city': 'South Jeremy', 'user_id': UUID('4a4a6d13-cec0-4544-92c6-4822b9d83cc5'), 'last_access': datetime.datetime(2026, 1, 10, 2, 4, 24, 684000), 'session_duration': 2060, 'username': 'ericjohnson'}
{'city': 'East Williamfurt', 'user_id': UUID('260e2b88-29e9-4875-bc1b-838d0a8515c1'), 'last_access': datetime.datetime(2026, 1, 10, 2, 4, 24, 674000), 'session_duration': 3004, 'username': 'bergdana'}
{'city': 'West Mikemouth', 'user_id': UUID('3ef06b55-f247-4d4a-97db-1cbf9e219c8a'), 'last_access': datetime.datetime(2026, 1, 10, 2, 4, 24, 667000), 'session_duration': 2543, 'username': 'bryanjones'}
{'city': 'Williamside', 'user_id': UUID('bb3c923a-04e0-447b-a711-00b9e8862ea2'), 'last_access': datetime.datetime(2026, 1, 10, 2, 4, 24, 673000), 'session_duration': 744, 'username': 'richardso

In [12]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'Lake Jacobtown'}
{'city': 'South Jeremy'}
{'city': 'East Williamfurt'}
{'city': 'West Mikemouth'}
{'city': 'Williamside'}
{'city': 'Weberstad'}
{'city': 'Reedtown'}
{'city': 'Penabury'}
{'city': 'Port Stacy'}
{'city': 'North Nicole'}
{'city': 'Sarahburgh'}
{'city': 'Whitneyton'}
{'city': 'West Michelle'}
{'city': 'Elizabethhaven'}
{'city': 'Kevinport'}
{'city': 'Deanmouth'}
{'city': 'East Stacey'}
{'city': 'Blackmouth'}
{'city': 'Keithside'}
{'city': 'Brittanybury'}
{'city': 'Smithfurt'}
{'city': 'Rogersstad'}
{'city': 'West Jenniferberg'}
{'city': 'New Ronaldport'}
{'city': 'South Christopher'}
{'city': 'Williambury'}
{'city': 'New Peter'}
{'city': 'Jeffreymouth'}
{'city': 'Angelaside'}
{'city': 'Gonzalezview'}
{'city': 'Lake Jacqueline'}
{'city': 'Griffinbury'}
{'city': 'Travisport'}
{'city': 'Annmouth'}
{'city': 'South Angelamouth'}
{'city': 'West Karamouth'}
{'city': 'East Coreychester'}
{'city': 'New Gina'}
{'city': 'Lynnmouth'}
{'city': 'Jesusport'}
{'city': 'Shannonvie

### Find nodes storing random data

In [13]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Williamstown',
 'last_access': datetime.datetime(2026, 1, 10, 2, 4, 24, 694000),
 'session_duration': 1123,
 'user_id': UUID('25a9f5ed-0e7c-4499-9de2-c73e2b368ce7'),
 'username': 'lpotter'}
Nodes storing 'Williamstown':
 - Host: 10.15.20.2, Gossip Port: 7003
 - Host: cassandra-node-1.mavasbel.vpn.itam.mx, Gossip Port: 7001
 - Host: 10.15.20.2, Gossip Port: 7002


### ORM-like

In [14]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [15]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=Joshuaside, user_id=d387f7ae-93a7-42a2-a3d4-61b2856aa3b3>
Query user_metris: UserMetrics <city=Lake Jacobtown, user_id=92886505-24aa-438b-8cae-770be6e38ee6>
Query user_metris: UserMetrics <city=South Jeremy, user_id=4a4a6d13-cec0-4544-92c6-4822b9d83cc5>
Query user_metris: UserMetrics <city=East Williamfurt, user_id=260e2b88-29e9-4875-bc1b-838d0a8515c1>
Query user_metris: UserMetrics <city=West Mikemouth, user_id=3ef06b55-f247-4d4a-97db-1cbf9e219c8a>
Query user_metris: UserMetrics <city=Williamside, user_id=bb3c923a-04e0-447b-a711-00b9e8862ea2>
Query user_metris: UserMetrics <city=Weberstad, user_id=3cfa1b0b-c659-47bc-8e8c-846f08b2cf57>
Query user_metris: UserMetrics <city=Reedtown, user_id=43ddc22e-d801-422d-8e11-e056b742407d>
Query user_metris: UserMetrics <city=Penabury, user_id=c8aa4d75-e276-4685-9c66-985fa1917bda>
Query user_metris: UserMetrics <city=Port Stacy, user_id=98e5048a-d610-436a-8017-712d8f5d5fde>
Query user_metris: UserMetrics <city=N

### Insert/Create

In [16]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='Larryborough', user_id=UUID('24778c19-7bfe-4ec0-8d36-37984c917d38'), username='gutierrezjonathan', session_duration=52, last_access=datetime.datetime(1989, 11, 2, 5, 26, 49))

In [17]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [18]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=Lake Jacobtown, user_id=92886505-24aa-438b-8cae-770be6e38ee6>
Delete user_metris: UserMetrics <city=South Jeremy, user_id=4a4a6d13-cec0-4544-92c6-4822b9d83cc5>
Delete user_metris: UserMetrics <city=East Williamfurt, user_id=260e2b88-29e9-4875-bc1b-838d0a8515c1>
Delete user_metris: UserMetrics <city=West Mikemouth, user_id=3ef06b55-f247-4d4a-97db-1cbf9e219c8a>
Delete user_metris: UserMetrics <city=Williamside, user_id=bb3c923a-04e0-447b-a711-00b9e8862ea2>
Delete user_metris: UserMetrics <city=South Ronaldland, user_id=94f5818b-b437-4970-85f5-b6803804e434>
Delete user_metris: UserMetrics <city=North Monica, user_id=42527625-3283-4d06-833d-f26a84194b1f>
Delete user_metris: UserMetrics <city=Weberstad, user_id=3cfa1b0b-c659-47bc-8e8c-846f08b2cf57>
Delete user_metris: UserMetrics <city=Reedtown, user_id=43ddc22e-d801-422d-8e11-e056b742407d>
Delete user_metris: UserMetrics <city=Port Jerryberg, user_id=033ca3b8-2e34-4155-af25-0e7450ad7435>
Delete user_me

In [19]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")